In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature = 0.1,
                  streaming=True,
                  callbacks=[
                      StreamingStdOutCallbackHandler(),
                  ]
                  )
# LangChain Expression Language (LCEL):

poem_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are Yun Dong-ju a famous poet in Korea. You create poem about program language."),
    ("human", "I want to make poem about {language}.")
])

poem_chain = poem_prompt | chat

In [4]:
get_poem_prompt = ChatPromptTemplate.from_messages([
    ("system", "You explain the poem in detail and explain with the six and a half principles what emotions and thoughts you had when you wrote it."),
    ("human", "{recieve}")
])

get_chain = get_poem_prompt | chat

final_chain = {"recieve" :poem_chain} | get_chain
final_chain.invoke({
    "language" : "JavaScripts"
})

In the realm of code, where logic intertwines,
There dances a language, bright and divine.
JavaScripts, a symphony of commands,
Weaving tales of websites and apps in our hands.

With functions and loops, it paints a scene,
Of dynamic pages, ever so keen.
From variables to arrays, it sings its song,
A melody of code, both elegant and strong.

In the world of the web, it reigns supreme,
A language of power, like a lucid dream.
So let us embrace this language of might,
And watch as our creations take flight.The poem you've written beautifully captures the essence of JavaScript and its significance in the world of coding and web development. It portrays JavaScript as a vibrant and powerful language that allows developers to create dynamic and engaging websites and applications.

Let's analyze the poem using the six and a half principles to understand the emotions and thoughts you might have had while writing it:

1. **Form**: The poem follows a consistent rhyme scheme and structure, with f

AIMessageChunk(content='The poem you\'ve written beautifully captures the essence of JavaScript and its significance in the world of coding and web development. It portrays JavaScript as a vibrant and powerful language that allows developers to create dynamic and engaging websites and applications.\n\nLet\'s analyze the poem using the six and a half principles to understand the emotions and thoughts you might have had while writing it:\n\n1. **Form**: The poem follows a consistent rhyme scheme and structure, with four stanzas each consisting of four lines. This structured form reflects a sense of order and organization, mirroring the logical nature of coding.\n\n2. **Imagery**: The poem uses vivid imagery to depict JavaScript as a "symphony of commands" and a "melody of code." These metaphors evoke a sense of beauty and creativity in the world of programming.\n\n3. **Sound**: The use of alliteration in phrases like "dynamic pages, ever so keen" and "melody of code, both elegant and str